In [1]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

import transformers
from datasets import load_dataset
import evaluate

from transformers import AutoTokenizer
import re

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


c:\Users\denis\anaconda3\envs\neuro-research-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
ds = load_dataset(
    "json",
    name="custom",  # Любое допустимое имя (без спецсимволов)
    data_files=r"C:\Users\denis\Downloads\val\val\a\data000000000000\data000000000000.json"
)

Generating train split: 56 examples [00:00, 344.42 examples/s]


In [34]:
ds['train'][1]

{'publication_number': 'US-4214630-A',
 'abstract': 'Nozzle for use with two different fire fighting agents, these being a ball valve and an exit tube for each agent.',
 'application_number': 'US-96611378-A',
 'description': 'BACKGROUND OF THE INVENTION \n     In fire trucks and similar apparatus, it is common practice to provide a plurality of fire fighting agents that may be used selectively, depending on the type of fire encountered. For instance, in some types of fire it is desirable to use a foam, whereas with other types it is desirable to use a powder. It is desirable to connect these two agents to a common nozzle, so that the selection of agent may be made at the point of use. The operator may wish to use the agents separately, as experience dictates, to extinguish the fire. It is on occasion desirable to use both agents at the same time. My U.S. Pat. No. 4,030,666, which issued on June 21, 1977, shows a construction for performing these functions in response to a trigger mecha

In [ ]:
for i in range(56):
    print(len((ds['train'][i]['abstract']).split()))

In [1]:
from datasets import load_dataset
import aiohttp
from datasets import disable_caching


disable_caching()

data = load_dataset("big_patent", "all", 
                     split="train", 
                     trust_remote_code=True,
                     streaming=True,
                     storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=3600)}}
                     )

d:\ethd\conda\envs\neuro-research-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ethd\conda\envs\neuro-research-env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\denis\.cache\huggingface\hub\datasets--big_patent. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer 

In [6]:
from datasets import load_dataset, concatenate_datasets
import aiohttp

arxiv = load_dataset("scientific_papers", "arxiv", split="train", trust_remote_code=True, storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=3600)}})
booksum = load_dataset("kmfoda/booksum", split="train", trust_remote_code=True, storage_options={'client_kwargs': {'timeout': aiohttp.ClientTimeout(total=3600)}})

# arxiv = arxiv.rename_column("article", "text")
# arxiv = arxiv.rename_column("abstract", "summary")

# booksum = booksum.rename_column("chapter", "text")
# booksum = booksum.rename_column("summary_text", "summary")

combined_dataset = concatenate_datasets([arxiv, booksum])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

def preprocess_function(examples):
    inputs = [re.sub(r'<[^>]+>|http\S+', '', text) for text in examples["article"]]
    targets = [re.sub(r'[\U00010000-\U0010ffff]', '', text) for text in examples["summary"]]
    
    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        add_special_tokens=True,
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation=True,
            add_special_tokens=True,
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    predict_with_generate=True,
    generation_max_length=128,  # Максимальная длина суммы
)

trainer = Seq2SeqTrainer(
    # model=model,
    args=training_args,
    # train_dataset=train_dataset,
    # eval_dataset=val_dataset,
)
trainer.train()